In [1]:
!pip3 install apache_beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 16.9 MB/s eta 0:00:00
  Created wheel for crcmod: filename=crcmod

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/Colab Notebooks/Cloud-AI-Analytics/Apache\ Beam\ -Python/data

/content/drive/MyDrive/Colab Notebooks/Cloud-AI-Analytics/Apache Beam -Python/data


In [7]:
!ls

dept_data.txt  regular_filter.txt-00000-of-00001  Students_age.txt	students.txt
grocery.txt    result				  students_exclude.txt


In [22]:
!{('head -n 30 dept_data.txt')}

149633CM,Marco,10,Accounts,1-01-2019
212539MU,Rebekah,10,Accounts,1-01-2019
231555ZZ,Itoe,10,Accounts,1-01-2019
503996WI,Edouard,10,Accounts,1-01-2019
704275DC,Kyle,10,Accounts,1-01-2019
957149WC,Kyle,10,Accounts,1-01-2019
241316NX,Kumiko,10,Accounts,1-01-2019
796656IE,Gaston,10,Accounts,1-01-2019
331593PS,Beryl,20,HR,1-01-2019
560447WH,Olga,20,HR,1-01-2019
222997TJ,Leslie,20,HR,1-01-2019
171752SY,Mindy,20,HR,1-01-2019
153636AS,Vicky,20,HR,1-01-2019
745411HT,Richard,20,HR,1-01-2019
298464HN,Kirk,20,HR,1-01-2019
783950BW,Kaori,20,HR,1-01-2019
892691AR,Beryl,20,HR,1-01-2019
245668UZ,Oscar,20,HR,1-01-2019
231206QD,Kumiko,30,Finance,1-01-2019
357919KT,Wendy,30,Finance,1-01-2019
472418ZG,Cristobal,30,Finance,1-01-2019
442292OI,Erika,30,Finance,1-01-2019
503647MN,Sebastien,30,Finance,1-01-2019
245319LD,Valerie,30,Finance,1-01-2019
818776XR,Dolly,30,Finance,1-01-2019
259229ZU,Emily,30,Finance,1-01-2019
349360YC,Kaori,30,Finance,1-01-2019
951594MT,Hitomi,30,Finance,1-01-2019
149633CM,Marco,10,

In [10]:
import apache_beam as beam

def SplitRow(element):
  return element.split(',')


def filter_on_count(element):
  name, count = element
  if count > 30:
    return element

def format_output(element):
  name, count = element
  return (name.encode('ascii'),str(count),'Experienced employee')

## Using normal transformation like



1.   Map
2.   Filter
3.   CombinePerKey

For three different operations and it includes more code, memory space and time as well.


In [27]:
with beam.Pipeline() as pipeline:
  input_data = (pipeline
            | "Read from text file" >> beam.io.ReadFromText('dept_data.txt')
            | "Split rows" >> beam.Map(SplitRow)
          )

  accounts_count = (input_data
              | 'Get all Accounts dept persons' >> beam.Filter(lambda record: record[3] == 'Accounts')
              | 'Pair each accounts employee with 1' >> beam.Map(lambda record: ("Accounts, " +record[1], 1))
              | 'Group and sum1' >> beam.CombinePerKey(sum)
              | 'count filter accounts' >> beam.Filter(filter_on_count)
              | 'Regular accounts employee' >> beam.Map(format_output)
              | 'Write results for account' >> beam.io.WriteToText('data/Account_quick')
            )

  hr_count =    (input_data
              | 'Get all HR dept persons' >> beam.Filter(lambda record: record[3] == 'HR')
              | 'Pair each HR employee with 1' >> beam.Map(lambda record: ("HR, " +record[1], 1))
              | 'HR Group and sum1' >> beam.CombinePerKey(sum)
              | 'HR count filter accounts' >> beam.Filter(filter_on_count)
              | 'HR Regular accounts employee' >> beam.Map(format_output)
              | 'Write results for HR' >> beam.io.WriteToText('data/HR_quick')
            )

  finance_count = (input_data
              | 'Get all Finance dept persons' >> beam.Filter(lambda record: record[3] == 'Finance')
              | 'Pair each finance employee with 1' >> beam.Map(lambda record: ("Finance, " +record[1], 1))
              | 'Finance Group and sum11' >> beam.CombinePerKey(sum)
              | 'Finance count filter accounts' >> beam.Filter(filter_on_count)
              | 'Finance Regular accounts employee' >> beam.Map(format_output)
              | 'Write results for finance' >> beam.io.WriteToText('data/Finance_quick')
           )

In [28]:
!{('head -n 10 data/Account_quick-00000-of-00001')}

(b'Accounts, Marco', '31', 'Experienced employee')
(b'Accounts, Rebekah', '31', 'Experienced employee')
(b'Accounts, Itoe', '31', 'Experienced employee')
(b'Accounts, Edouard', '31', 'Experienced employee')
(b'Accounts, Kyle', '62', 'Experienced employee')
(b'Accounts, Kumiko', '31', 'Experienced employee')
(b'Accounts, Gaston', '31', 'Experienced employee')


In [29]:
!{('head -n 10 data/HR_quick-00000-of-00001')}

(b'HR, Beryl', '62', 'Experienced employee')
(b'HR, Olga', '31', 'Experienced employee')
(b'HR, Leslie', '31', 'Experienced employee')
(b'HR, Mindy', '31', 'Experienced employee')
(b'HR, Vicky', '31', 'Experienced employee')
(b'HR, Richard', '31', 'Experienced employee')
(b'HR, Kirk', '31', 'Experienced employee')
(b'HR, Kaori', '31', 'Experienced employee')
(b'HR, Oscar', '31', 'Experienced employee')


In [30]:
!{('head -n 10 data/Finance_quick-00000-of-00001')}

(b'Finance, Kumiko', '31', 'Experienced employee')
(b'Finance, Wendy', '31', 'Experienced employee')
(b'Finance, Cristobal', '31', 'Experienced employee')
(b'Finance, Erika', '31', 'Experienced employee')
(b'Finance, Sebastien', '31', 'Experienced employee')
(b'Finance, Valerie', '31', 'Experienced employee')
(b'Finance, Dolly', '31', 'Experienced employee')
(b'Finance, Emily', '31', 'Experienced employee')
(b'Finance, Kaori', '31', 'Experienced employee')
(b'Finance, Hitomi', '31', 'Experienced employee')


## **Composite Transforms:**

* Transforms can have a nested structure, where a complex transform performs multiple simpler transforms (such as more than one ParDo, Combine, GroupByKey, or even other composite transforms). These transforms are called composite transforms.

In [23]:
class MyTransform(beam.PTransform):

  def expand(self, input_coll):

    a = (
        input_coll
        | 'Group and sum1' >> beam.CombinePerKey(sum)
        | 'count filter accounts' >> beam.Filter(filter_on_count)
        | 'Regular accounts employee' >> beam.Map(format_output)

    )
    return a

In [24]:
with beam.Pipeline() as p:
  input_data = (p
            | "Read from text file" >> beam.io.ReadFromText('dept_data.txt')
            | "Split rows" >> beam.Map(SplitRow)
          )

  accounts_count = (
              input_data
              | 'Get all Accounts dept persons' >> beam.Filter(lambda record: record[3] == 'Accounts')
              | 'Pair each accounts employee with 1' >> beam.Map(lambda record: ("Accounts, " +record[1], 1))
              | 'composite accounts' >> MyTransform()
              | 'Write results for account' >> beam.io.WriteToText('data/Account')
            )

  finance_count = (
              input_data
              | 'Get all Finance dept persons' >> beam.Filter(lambda record: record[3] == 'Finance')
              | 'Pair each Finance employee with 1' >> beam.Map(lambda record: ("Finance, " +record[1], 1))
              | 'composite Finance' >> MyTransform()
              | 'Write results for Finance' >> beam.io.WriteToText('data/Finance')
           )

In [25]:
!{('head -n 10 data/Account-00000-of-00001')}

(b'Accounts, Marco', '31', 'Experienced employee')
(b'Accounts, Rebekah', '31', 'Experienced employee')
(b'Accounts, Itoe', '31', 'Experienced employee')
(b'Accounts, Edouard', '31', 'Experienced employee')
(b'Accounts, Kyle', '62', 'Experienced employee')
(b'Accounts, Kumiko', '31', 'Experienced employee')
(b'Accounts, Gaston', '31', 'Experienced employee')


In [26]:
!{('head -n 10 data/Finance-00000-of-00001')}

(b'Finance, Kumiko', '31', 'Experienced employee')
(b'Finance, Wendy', '31', 'Experienced employee')
(b'Finance, Cristobal', '31', 'Experienced employee')
(b'Finance, Erika', '31', 'Experienced employee')
(b'Finance, Sebastien', '31', 'Experienced employee')
(b'Finance, Valerie', '31', 'Experienced employee')
(b'Finance, Dolly', '31', 'Experienced employee')
(b'Finance, Emily', '31', 'Experienced employee')
(b'Finance, Kaori', '31', 'Experienced employee')
(b'Finance, Hitomi', '31', 'Experienced employee')
